# Deep Voice 2: Multi-Speaker Neural Text-to-Speech

Link: http://research.baidu.com/wp-content/uploads/2017/05/Deep-Voice-2-Complete-Arxiv.pdf

Authors: Sercan O. Arık, Andrew Gibiansky, Wei Ping, Gregory Diamos, John Miller, Jonathan Raiman, Kainan Peng, Yanqi Zhou

Institution: Baidu Silicon Valley Artificial Intelligence Lab

Publication: 31st Conference on Neural Information Processing Systems (NIPS 2017)

Date: 2017



## Background Materials

- http://research.baidu.com/deep-voice-2-multi-speaker-neural-text-speech/


## What is this paper about?




## What is the motivation of this research?




## What makes this paper different from previous research?




## How this paper achieve it?




## Dataset used in this study




## Implementations




## Further Readings